In [ ]:
import ROOT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import math
import uproot
from array import array

print("Got imports")

In [ ]:
DIM = 0


data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/Widths1D_V0/data_0.root"
#mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/Widths1D_V0/mc_0.root"

#mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/Widths1D_V0/mc_0_modded_test.root"
mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/Widths1D_V0/mc_0_modded_large_26Aug2025_std_err.root"



#data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/Widths1D_V0/data_1.root"
#mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/Widths1D_V0/mc_1.root"

#data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/Widths1D_V1/data_6.root"
#mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/Widths1D_V1/mc_6.root"


# Residual Range with Stopping Muons
#data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/data_stopping_6.root"
#mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Fits/mc_stopping_6.root"


print("Got MC and Data Files to compare and analyze")

In [ ]:
rfile_data = ROOT.TFile.Open(data_file)
rfile_mc = ROOT.TFile.Open(mc_file)

DATA_TAG = "v10_06_00_02"

rfile_data.ls()

# Helpers

In [ ]:
# I want to convert a ROOT TH2D to a plt.hist2d for better plotting
def root_th2d_to_plt_hist2d(th2d):
    # Get the bin edges and contents from the ROOT TH2D
    x_edges = [th2d.GetXaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsX() + 2)]
    y_edges = [th2d.GetYaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsY() + 2)]
    z_values = np.zeros((th2d.GetNbinsX(), th2d.GetNbinsY()))

    for i in range(1, th2d.GetNbinsX() + 1):
        for j in range(1, th2d.GetNbinsY() + 1):
            z_values[i-1, j-1] = th2d.GetBinContent(i, j)

    return np.array(x_edges), np.array(y_edges), np.array(z_values)

def get_1d_edges(h):
    x_edges = [h.GetXaxis().GetBinLowEdge(i) for i in range(1, h.GetNbinsX() + 2)] 
    return x_edges

# Example usage:
# th2d = rfile_data.Get("hwidth0")
# x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
# plt.hist2d(x_edges[:-1], y_edges[:-1], z_values.T, bins=[x_edges, y_edges], cmap='viridis')
# plt.colorbar(label='Counts')
# plt.xlabel('X-axis label')                
#plt.ylabel('Y-axis label')
# plt.title('2D Histogram from ROOT TH2D')
# plt.show()    

# I want to convert a ROOT TH1D with errors to a plt.errorbar for better plotting
def root_th1d_to_plt_errorbar(th1d):
    # Get the bin centers, contents, and errors from the ROOT TH1D
    x_values = [th1d.GetBinCenter(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_values = [th1d.GetBinContent(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_errors = [th1d.GetBinError(i) for i in range(1, th1d.GetNbinsX() + 1)]

    return np.array(x_values), np.array(y_values), np.array(y_errors) 

# Example usage:
# th1d = rfile_data.Get("hwidth0").ProjectionX()
# x_values, y_values, y_errors = root_th1d_to_plt_errorbar(th1d)
# plt.errorbar(x_values, y_values, yerr=y_errors, fmt='o', ecolor='red', capsize=5)
# plt.xlabel('X-axis label')                
# plt.ylabel('Y-axis label')
# plt.title('1D Histogram with Error Bars from ROOT TH1D')
# plt.show()


# Function to create a ROOT TH1D from means and errors
def make_mean_hist(h2d, means, errors):
    h_means = h2d.ProjectionX()
    h_means.Reset()
    for i in range(1, h_means.GetNbinsX() + 1):
        h_means.SetBinContent(i, means[i-1])
        h_means.SetBinError(i, errors[i-1])
    
    return h_means

In [ ]:
dim_dict = {0: "x", 1: "y", 2: "z", 3:"txz", 4:"tyz", 5:"dqdx"}
label_dict = {0: "Reconstructed Hit X [cm]", 
              1: "Reconstructed Hit Y [cm]", 
              2: "Reconstructed Hit Z [cm]", 
              3:"Reconstructed Hit "+r"$\theta_{XZ}$ [degrees]", 
              4:"Reconstructed Hit "+r"$\theta_{YZ}$ [degrees]", 
              5:"Reconstructed Hit dQ/dx [ADC/cm]",
              6:"Residual Range [cm]"}


xlim_dict = {0:[[-200, 0], [0, 200]],
             1: [[-200, 200], [-200, 200]],
             6:[[0, 400], [0, 400]]

}


In [ ]:

def plot_fits(DIM, xlim0=None, xlim1=None, ylim=None, DATA_TAG=None):

    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    gs = gridspec.GridSpec(1, 2 + 1, width_ratios=[1, 1, 0.05], wspace=0.1)

    largest_max_z = 0

    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane
            #thn = rfile_data.Get("hwidth"+str(idx))
            h = 0
            
            h_data = rfile_data.Get("h"+str(idx)+"_"+str(DIM))
            h_mc = rfile_mc.Get("h"+str(idx)+"_"+str(DIM))

            x_edges = get_1d_edges(h_data)

            ax = axs[tpc, plane]
            
            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)

            if ylim is not None:
                ax.set_ylim(ylim)

            ax.set_title(f'TPC {tpc}, Plane {plane}')


            x, y, yerr = root_th1d_to_plt_errorbar(h_data)
            x_mc, y_mc, yerr_mc = root_th1d_to_plt_errorbar(h_mc)

            ax.errorbar(x, y, xerr=np.ones_like(yerr)*((x_edges[1]-x_edges[0])/2), 
                        yerr=yerr, fmt='o', ecolor='black', capsize=0, label='Data', markersize=0)
            ax.errorbar(x_mc, y_mc, xerr=np.ones_like(yerr_mc)*((x_edges[1]-x_edges[0])/2), 
                        yerr=yerr_mc, fmt='o', ecolor='red', capsize=0, label='MC', markersize=0)
            
            ax.legend()
            
            h_data.Delete()
            h_mc.Delete()

   
            

    fig.supxlabel(label_dict[DIM], fontsize=20)
    fig.supylabel("Reconstructed Hit Width [ticks]", fontsize=20)
    t = "SBND Off-Beam Calibration: "

    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    #cbar_ax = fig.add_subplot(gs[0, 2])
    #fig.colorbar(pcm, cax=cbar_ax)
    #fig.colorbar(pcm, ax=axs, fraction=.05, label='')
    plt.tight_layout()
    plt.show()

print("Defining plot function")

In [ ]:
plot_fits(DIM, xlim0=xlim_dict[DIM][0], xlim1=xlim_dict[DIM][1], ylim=[3.4, 3.8], DATA_TAG=DATA_TAG)

In [ ]:

def plot_ratio_fits(DIM, xlim0=None, xlim1=None, ylim=None, DATA_TAG=None):

    fit_dict = {}
    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    gs = gridspec.GridSpec(1, 2 + 1, width_ratios=[1, 1, 0.05], wspace=0.1)

    largest_max_z = 0

    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane
            #thn = rfile_data.Get("hwidth"+str(idx))
            h = 0
            
            h_data = rfile_data.Get("h"+str(idx)+"_"+str(DIM))
            h_mc = rfile_mc.Get("h"+str(idx)+"_"+str(DIM))

            x_edges = get_1d_edges(h_data)

            x, y, yerr = root_th1d_to_plt_errorbar(h_data)
            x_mc, y_mc, yerr_mc = root_th1d_to_plt_errorbar(h_mc)

            # SPLINE STUFF ----------------------------------- /
            bin_centers = []
            for bin in range(1, h_data.GetNbinsX() + 1):
                bin_centers.append(h_data.GetBinCenter(bin))
            bin_centers = np.array(bin_centers)
            ratios = y / y_mc
            #ratio_errors = np.sqrt((errors_data / means_mc)**2 + (means_data * errors_mc / means_mc**2)**2)
            ratio_errors = ratios*np.sqrt((yerr/y)**2 + (yerr_mc/y_mc)**2)
            
            spline_ratios = None
            x_vals = None
            # Fit a spline to the ratios in the region of non-zero values
            non_zero_indices = np.where(ratios > 0)[0]
            if len(non_zero_indices) > 0:
                m = 0
                if xlim0 is not None and tpc == 0:
                    m = (bin_centers > xlim0[0]) & (bin_centers < xlim0[1])
                elif xlim1 is not None and tpc == 1:
                    m = (bin_centers > xlim1[0]) & (bin_centers < xlim1[1])

                ratios_for_fit = ratios[m]
                from scipy.interpolate import UnivariateSpline
                from scipy.interpolate import BSpline 
                #spline = UnivariateSpline(bin_centers[m], ratios_for_fit, s=0.01)
                spline = UnivariateSpline(bin_centers[m], ratios_for_fit, k=3)
                tck = spline._eval_args
                t, c, k = tck
                #spline = make_splrep(bin_centers[m], ratios_for_fit, s=0.01)
                print("Index", idx)
                print("Knots:", spline.get_knots())
                print("Knots2:", t)
                print("Coefficients:", spline.get_coeffs())
                print("Coefficients2:", c)
                print("Degree", k)

                #fit_dict[idx] = [spline.get_knots(), spline.get_coeffs()]
                fit_dict[idx] = [t, c]
                #print("Degree:", spline.get_degree())
                # degree (order of spline)
                #print("Degree:", spline.k)
                print("")
                w = bin_centers[1] - bin_centers[0]
                x_vals = np.linspace(bin_centers[m][0], bin_centers[m][-1], 1000)
                spline_ratios = spline(x_vals)


            ax = axs[tpc, plane]
            
            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
                    ax.plot(xlim0, [1, 1], c="black", linestyle="--", linewidth=2)
            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)
                    ax.plot(xlim1, [1, 1], c="black", linestyle="--", linewidth=2)

            if ylim is not None:
                ax.set_ylim(ylim)

            ax.set_title(f'TPC {tpc}, Plane {plane}')

            ax.errorbar(bin_centers, ratios, xerr=np.ones_like(bin_centers)*((bin_centers[1]-bin_centers[0])/2), 
                yerr=np.zeros_like(ratios), fmt='o', ecolor='red', capsize=0, label='Iterative Truncated Mean', markersize=0)
            
            ax.plot(x_vals, spline_ratios, c='blue', label='Spline Fit', linewidth=2, linestyle='--')

            ax.legend()
            
            h_data.Delete()
            h_mc.Delete()
   
    

    fig.supxlabel(label_dict[DIM], fontsize=20)
    fig.supylabel("Data/Simulation (Hit Widths)", fontsize=20)
    t = "SBND Off-Beam Calibration: "

    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    #cbar_ax = fig.add_subplot(gs[0, 2])
    #fig.colorbar(pcm, cax=cbar_ax)
    #fig.colorbar(pcm, ax=axs, fraction=.05, label='')
    plt.tight_layout()
    plt.show()
    
    return fit_dict

print("Defining plot function")

In [ ]:
fits = plot_ratio_fits(DIM, xlim0=xlim_dict[DIM][0], xlim1=xlim_dict[DIM][1], ylim=[0.95, 1.05], DATA_TAG=DATA_TAG)

In [ ]:

def plot_splines(fits, DIM, xlim0=None, xlim1=None, ylim=None, DATA_TAG=None):

    
    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    gs = gridspec.GridSpec(1, 2 + 1, width_ratios=[1, 1, 0.05], wspace=0.1)

    largest_max_z = 0

    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane
            #thn = rfile_data.Get("hwidth"+str(idx))
            h = 0
            
            h_data = rfile_data.Get("h"+str(idx)+"_"+str(DIM))
            h_mc = rfile_mc.Get("h"+str(idx)+"_"+str(DIM))

            x_edges = get_1d_edges(h_data)

            x, y, yerr = root_th1d_to_plt_errorbar(h_data)
            x_mc, y_mc, yerr_mc = root_th1d_to_plt_errorbar(h_mc)

            # SPLINE STUFF ----------------------------------- /
            bin_centers = []
            for bin in range(1, h_data.GetNbinsX() + 1):
                bin_centers.append(h_data.GetBinCenter(bin))
            bin_centers = np.array(bin_centers)
            ratios = y / y_mc
            #ratio_errors = np.sqrt((errors_data / means_mc)**2 + (means_data * errors_mc / means_mc**2)**2)
            ratio_errors = ratios*np.sqrt((yerr/y)**2 + (yerr_mc/y_mc)**2)
            
            spline_ratios = None
            x_vals = None

            current_fit = fits[idx]

            # Fit a spline to the ratios in the region of non-zero values
            non_zero_indices = np.where(ratios > 0)[0]
            if len(non_zero_indices) > 0:
                m = 0
                if xlim0 is not None and tpc == 0:
                    m = (bin_centers > xlim0[0]) & (bin_centers < xlim0[1])
                elif xlim1 is not None and tpc == 1:
                    m = (bin_centers > xlim1[0]) & (bin_centers < xlim1[1])
 
                from scipy.interpolate import BSpline 
                #spline = UnivariateSpline(bin_centers[m], ratios_for_fit, s=0.01)
                spline = BSpline(current_fit[0], current_fit[1], 3)

                w = bin_centers[1] - bin_centers[0]
                x_vals = np.linspace(bin_centers[m][0], bin_centers[m][-1], 1000)
                spline_ratios = spline(x_vals)


            ax = axs[tpc, plane]
            
            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
                    ax.plot(xlim0, [1, 1], c="black", linestyle="--", linewidth=2)
            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)
                    ax.plot(xlim1, [1, 1], c="black", linestyle="--", linewidth=2)

            if ylim is not None:
                ax.set_ylim(ylim)

            ax.set_title(f'TPC {tpc}, Plane {plane}')

            ax.errorbar(bin_centers, ratios, xerr=np.ones_like(bin_centers)*((bin_centers[1]-bin_centers[0])/2), 
                yerr=np.zeros_like(ratios), fmt='o', ecolor='red', capsize=0, label='Iterative Truncated Mean', markersize=0)
            
            ax.plot(x_vals, spline_ratios, c='blue', label='Spline Fit', linewidth=2, linestyle='--')

            ax.legend()
            
            h_data.Delete()
            h_mc.Delete()
   
    

    fig.supxlabel(label_dict[DIM], fontsize=20)
    fig.supylabel("Data/Simulation (Hit Widths)", fontsize=20)
    t = "SBND Off-Beam Calibration: "

    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    #cbar_ax = fig.add_subplot(gs[0, 2])
    #fig.colorbar(pcm, cax=cbar_ax)
    #fig.colorbar(pcm, ax=axs, fraction=.05, label='')
    plt.tight_layout()
    plt.show()
    
 

print("Defining plot function")

In [ ]:
plot_splines(fits, DIM, xlim0=xlim_dict[DIM][0], xlim1=xlim_dict[DIM][1], ylim=[0.95, 1.05], DATA_TAG=DATA_TAG)

In [ ]:
with open("spline_fits_1D_DIM_"+dim_dict[DIM]+".txt", "a") as f:  # "a" to append multiple fits
    for num in range(len(fits.keys())):
        f.write(f"Index {num}\n")
        f.write(f"k = {3}\n")
        f.write("t = " + " ".join(map(str, fits[num][0])) + "\n")
        f.write("c = " + " ".join(map(str, fits[num][1])) + "\n")
        f.write("\n")  # blank line between fits

print("Saved Fit results to a text file")